# Doublet Detection Tutorial

Author: [Hui Ma](https://github.com/huimalinda)<br>
Date: 2020-12-16<br>
Notebook Source: [doublet_detection.ipynb](https://raw.githubusercontent.com/klarman-cell-observatory/pegasus/master/notebooks/doublet_detection.ipynb)

In [ ]:
import pegasus as pg

## Dataset
In this tutorial, we'll use the output result of [Pegasus Tutorial](https://pegasus.readthedocs.io/en/latest/_static/tutorials/pegasus_analysis.html) to demonstrate how to detect and remove doublet cells in Pegasus. The dataset consists of human bone marrow single cells from 8 donors.

The dataset is stored at https://storage.googleapis.com/terra-featured-workspaces/Cumulus/MantonBM_result.zarr.zip. You can also use `gsutil` to download it via its Google bucket URL (gs://terra-featured-workspaces/Cumulus/MantonBM_result.zarr.zip).

Now load the count matrix:

In [ ]:
data = pg.read_input("MantonBM_result.zarr.zip")
data

# Sections
-  [Detect Doublets](#mark)
-  [Remove Doublets and Recluster](#recluster)

## Detect Doublets
<a id='mark'></a>
In this step, infer doublets per channel. Set <b>clust_attr = 'anno'</b> to see the doublet density in each cluster and infer doublet cluster.

Also set <b>robust=True</b> to use robust PCA for reproducibility reason.

The method used for detecting doublets can be found [here](https://github.com/klarman-cell-observatory/pegasus/raw/master/doublet_detection.pdf).

In [ ]:
pg.infer_doublets(data, channel_attr = 'Channel', clust_attr = 'anno', robust = True) 

Here, plot annotation and Scrublet-like doublet score.

In [ ]:
pg.scatter(data,attrs=['anno','doublet_score'], basis='umap', wspace=1.2) 

We also want to see the doublet percentage of each cluster to decide if there is a doublet cluster.

In [ ]:
data.uns['pred_dbl_cluster']

All clusters have doublet percentage under 5%, so no need to mark any doublet clusters here. If any cluster has doublet percentage more than $50\%$, we can consider to mark it as doublet cluster. 

For example, If we want to mark 'CD14+ Monocyte' and 'CD14+ Monocyte-2' as doublet clusters, use command  
<b>pg.mark_doublets(data, dbl_clusts = 'anno:CD14+ Monocyte,CD14+ Monocyte-2')</b>

The <b>pg.mark_doublets</b> command will mark doublet cluster (if any), and write singlet/doublet assignment to the "demux_type" column attribute in data.obs. The "demux_type" attribute is also used for singlet/doublet assignment of cell hashing, nucleus hashing and genetics pooling data (see [documentation](https://cumulus.readthedocs.io/en/latest/demultiplexing.html)).

For this demonstration dataset, among 35,465 cells, 748 doublets detected. Doublet rate is $2.11\%$

In [ ]:
pg.mark_doublets(data)
data.obs['demux_type'].value_counts()

Doublets distribution can be better observed in umap plot.

In [ ]:
pg.scatter(data, attrs = ['anno', 'demux_type'], legend_loc = ['on data', 'right margin'], 
           wspace = 0.1,alpha = [1.0, 0.8], palettes = 'demux_type:gainsboro,red')

## Remove Doublets and Recluster
<a id='recluster'></a>

In [ ]:
pg.qc_metrics(data,select_singlets=True)
pg.filter_data(data)

Start the reclustering process from re-selecting highly variable genes. Batch effect is observed, so we also want to use harmony algorithm to correct bach effect for reclustering.

In [ ]:
pg.highly_variable_features(data, consider_batch=True)
pg.pca(data, robust=True)
pca_key = pg.run_harmony(data)
pg.neighbors(data,rep=pca_key)
pg.louvain(data,rep=pca_key)
pg.umap(data,rep=pca_key)

Re-annotate:

In [ ]:
pg.de_analysis(data, cluster='louvain_labels')
celltype_dict = pg.infer_cell_types(data, markers = 'human_immune',de_test='mwu',output_file='BM_celltype_re_dict.txt')
cluster_names = pg.infer_cluster_names(celltype_dict)
pg.annotate(data, name='anno', based_on='louvain_labels', anno_dict=cluster_names)

Umap of annotation after re-clustering:

In [ ]:
pg.scatter(data,attrs='anno',legend_loc='on data',basis='umap')